<a href="https://colab.research.google.com/github/aSafarpoor/Seminar/blob/main/NCF_Amazon_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**A pytorch GPU implementation of He et al. "Neural Collaborative Filtering"**

based on https://github.com/guoyang9/NCF

In [ ]:
batch_size=256 
lr=0.001 
factor_num=16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/MSc/codes

/content/drive/MyDrive/MSc/codes


# amazon data preparetion as input

In [ ]:
import json
from tqdm import tqdm
import random
from collections import Counter
import pandas as pd
import pickle
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd ..
%cd ..
# %cd ..
# %cd ..
# %cd ..
# %cd ..
%cd MyDrive
%cd Seminar
!ls

/content/drive/My Drive/MSc
/content/drive/My Drive
[Errno 2] No such file or directory: 'MyDrive'
/content/drive/My Drive
/content/drive/My Drive/Seminar
AMAZON_FASHION.json	     ml-1m.test.negative  runs
AMAZON_FASHION.json.gz	     ml-1m.test.rating	  test.csv
embedded_text.p		     ml-1m.train.rating   train.csv
meta_AMAZON_FASHION.json     model.bin		  train.gsheet
meta_AMAZON_FASHION.json.gz  model_plot.png	  user_item_matrix.p
meta_data.p		     models		  user_tweets.p


In [ ]:
tweets = []
for line in open('AMAZON_FASHION.json', 'r'):
    tweets.append(json.loads(line))

In [ ]:
elemans = []
udic = {}
idic = {}
for tweet in tqdm(tweets):
    uid = tweet['reviewerID']
    iid = tweet['asin']

    if uid in udic:
        uid = udic[uid]
    else:
        udic[uid] = len(udic)
        uid = udic[uid]

    if iid in idic:
        iid = idic[iid]
    else:
        idic[iid] = len(idic)
        iid = idic[iid]

    eleman = {'user':uid,'item':iid}
    elemans.append(eleman)

100%|██████████| 883636/883636 [00:02<00:00, 339779.75it/s]


In [ ]:
elemans = pd.DataFrame(elemans)
elemans


,user,item
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
883631,749229,186186
883632,749230,186186
883633,6782,186186
883634,749231,186187


In [ ]:
number_of_users = max(elemans.user),
number_of_items = max(elemans.item)

In [ ]:
len(elemans)

883636

In [ ]:
np.random.shuffle(elemans.values)
elemans

,user,item
0,386870,26478
1,546384,88960
2,641837,131638
3,670429,145830
4,597688,111353
...,...,...
883631,474635,59190
883632,123255,2754
883633,23618,193
883634,495360,5330


In [ ]:
elemans1 = elemans.iloc[:1000, :] #int(0.7*len(elemans))
elemans2 = elemans.iloc[1000:2000, :]

In [ ]:
elemans2

,user,item
1000,514698,75212
1001,207503,95600
1002,329789,171293
1003,332747,11359
1004,61298,944
...,...,...
1995,83550,1485
1996,655648,138461
1997,419108,37709
1998,244715,7293


In [ ]:
elemans1.to_csv('train.csv', sep ='\t', index=False)
elemans2.to_csv('test.csv', sep ='\t', index=False)

In [ ]:
train = pd.read_csv('train.csv',sep='\t', names=['user', 'item'],usecols=[0,1], index_col=False)
train = train.iloc[1:]

test = pd.read_csv('test.csv',sep='\t', names=['user', 'item'],usecols=[0,1], index_col=False)
test = test.iloc[1:]

In [ ]:
# train.user.apply(int)
# train.item.apply(int)

In [ ]:
# %cd MSc/codes/

In [ ]:
# # %mkdir recommendation
# %cd recommendation
# # %mkdir NCF-Data
# %cd NCF-Data
# !pwd

In [ ]:
# !wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.train.rating
# !wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.test.rating
# !wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.test.negative

In [ ]:
# df = pd.read_csv('ml-1m.test.negative',
#                              sep='\t', header = None, 
#                              index_col=False)
# df.head()

# Config

In [ ]:
class Config:
    def __init__(self):
        # dataset name 
        # self.dataset = 'ml-1m'

        # model name 
        self.model = 'NeuMF-end'

        # paths
        self.main_path = '/Seminar/'

        self.train_rating = self.main_path + 'train.csv'
        self.test_rating = self.main_path + 'test.csv'
        self.test_negative = self.main_path + 'test.negative.csv'

        self.model_path = './models/'
        self.GMF_model_path = self.model_path + 'GMF.pth'
        self.MLP_model_path = self.model_path + 'MLP.pth'
        self.NeuMF_model_path = self.model_path + 'NeuMF.pth'

# Data utils:

In [ ]:
import numpy as np 
import pandas as pd 
import scipy.sparse as sp
import torch.utils.data as data

config = Config()

In [ ]:
import random

In [ ]:

def load_all(test_num=100):
    ''' We load all the three file here to save time in each epoch. '''
    train_data = pd.read_csv('train.csv',
                             sep='\t', names=['user', 'item'],usecols=[0,1], 
                             index_col=False)
    train_data = train_data.iloc[1:]
    train_data['user'] = train_data['user'].apply(int)
    train_data['item'] = train_data['item'].apply(int)
    user_num = train_data['user'].max() + 1
    item_num = train_data['item'].max() + 1

    train_data = train_data.values.tolist()

    # load ratings as a dok matrix
    train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32)
    
    for x in tqdm(train_data):
        train_mat[x[0], x[1]] = 1.0

    test_data = []

    '''
    with open('/content/drive/MyDrive/Seminar' + config.test_negative, 'r') as fd:
        line = fd.readline() #(0,25)	106
        while line != None and line != '':
            arr = line.split('\t')
            u = eval(arr[0])[0]
            test_data.append([u, eval(arr[0])[1]])
            for i in arr[1:]:
                test_data.append([u, int(i)])
            line = fd.readline()'''

    test_data = pd.read_csv('test.csv',
                             sep='\t', names=['user', 'item'],usecols=[0,1], 
                             index_col=False)
    test_data = test_data.iloc[1:]
    test_data['user'] = test_data['user'].apply(int)
    test_data['item'] = test_data['item'].apply(int)

    test_data = test_data.values.tolist()

    # new_test_data = []
    # for d in tqdm(test_data):
    #     c = 4
    #     while(c!=0):
    #         rand_ = random.randint(0,number_of_items)
    #         if [d[0],rand_] in test_data or [d[0],rand_] in train_data:
    #             pass
    #         else:
    #             c-=1



    return train_data, test_data, user_num, item_num, train_mat		

train_data, test_data, user_num, item_num, train_mat = load_all()

100%|██████████| 1000/1000 [00:00<00:00, 77265.93it/s]


In [ ]:
class NCFData(data.Dataset):
	def __init__(self, features, 
				num_item, train_mat=None, num_ng=0, is_training=None):
		super(NCFData, self).__init__()
		""" Note that the labels are only useful when training, we thus 
			add them in the ng_sample() function.
		"""
		self.features_ps = features
		self.num_item = num_item
		self.train_mat = train_mat
		self.num_ng = num_ng
		self.is_training = is_training
		self.labels = [0 for _ in range(len(features))]

	def ng_sample(self):
		assert self.is_training, 'no need to sampling when testing'

		self.features_ng = []
		for x in self.features_ps:
			u = x[0]
			for t in range(self.num_ng):
				j = np.random.randint(self.num_item)
				while (u, j) in self.train_mat:
					j = np.random.randint(self.num_item)
				self.features_ng.append([u, j])

		labels_ps = [1 for _ in range(len(self.features_ps))]
		labels_ng = [0 for _ in range(len(self.features_ng))]

		self.features_fill = self.features_ps + self.features_ng
		self.labels_fill = labels_ps + labels_ng

	def __len__(self):
		return (self.num_ng + 1) * len(self.labels)

	def __getitem__(self, idx):
		features = self.features_fill if self.is_training \
					else self.features_ps
		labels = self.labels_fill if self.is_training \
					else self.labels

		user = features[idx][0]
		item = features[idx][1]
		label = labels[idx]
		return user, item ,label

# evaluate

In [ ]:
import numpy as np
import torch

In [ ]:
def hit(gt_item, pred_items):
	if gt_item in pred_items:
		return 1
	return 0

In [ ]:
def ndcg(gt_item, pred_items):
	if gt_item in pred_items:
		index = pred_items.index(gt_item)
		return np.reciprocal(np.log2(index+2))
	return 0

In [ ]:
def metrics(model, test_loader, top_k):
	HR, NDCG = [], []

	for user, item, label in test_loader:
		user = user.cuda()
		item = item.cuda()

		predictions = model(user, item)
		_, indices = torch.topk(predictions, top_k)
		recommends = torch.take(
				item, indices).cpu().numpy().tolist()

		gt_item = item[0].item()
		HR.append(hit(gt_item, recommends))
		NDCG.append(ndcg(gt_item, recommends))

	return np.mean(HR), np.mean(NDCG)

# model:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

In [ ]:
class NCF(nn.Module):
	def __init__(self, user_num, item_num, factor_num, num_layers,
					dropout, model, GMF_model=None, MLP_model=None):
		super(NCF, self).__init__()
		"""
		user_num: number of users;
		item_num: number of items;
		factor_num: number of predictive factors;
		num_layers: the number of layers in MLP model;
		dropout: dropout rate between fully connected layers;
		model: 'MLP', 'GMF', 'NeuMF-end', and 'NeuMF-pre';
		GMF_model: pre-trained GMF weights;
		MLP_model: pre-trained MLP weights.
		"""		
		self.dropout = dropout
		self.model = model
		self.GMF_model = GMF_model
		self.MLP_model = MLP_model

		self.embed_user_GMF = nn.Embedding(user_num, factor_num)
		self.embed_item_GMF = nn.Embedding(item_num, factor_num)
		self.embed_user_MLP = nn.Embedding(
				user_num, factor_num * (2 ** (num_layers - 1)))
		self.embed_item_MLP = nn.Embedding(
				item_num, factor_num * (2 ** (num_layers - 1)))

		MLP_modules = []
		for i in range(num_layers):
			input_size = factor_num * (2 ** (num_layers - i))
			MLP_modules.append(nn.Dropout(p=self.dropout))
			MLP_modules.append(nn.Linear(input_size, input_size//2))
			MLP_modules.append(nn.ReLU())
		self.MLP_layers = nn.Sequential(*MLP_modules)

		if self.model in ['MLP', 'GMF']:
			predict_size = factor_num 
		else:
			predict_size = factor_num * 2
		self.predict_layer = nn.Linear(predict_size, 1)

		self._init_weight_()

	def _init_weight_(self):
		""" We leave the weights initialization here. """
		if not self.model == 'NeuMF-pre':
			nn.init.normal_(self.embed_user_GMF.weight, std=0.01)
			nn.init.normal_(self.embed_user_MLP.weight, std=0.01)
			nn.init.normal_(self.embed_item_GMF.weight, std=0.01)
			nn.init.normal_(self.embed_item_MLP.weight, std=0.01)

			for m in self.MLP_layers:
				if isinstance(m, nn.Linear):
					nn.init.xavier_uniform_(m.weight)
			nn.init.kaiming_uniform_(self.predict_layer.weight, 
									a=1, nonlinearity='sigmoid')

			for m in self.modules():
				if isinstance(m, nn.Linear) and m.bias is not None:
					m.bias.data.zero_()
		else:
			# embedding layers
			self.embed_user_GMF.weight.data.copy_(
							self.GMF_model.embed_user_GMF.weight)
			self.embed_item_GMF.weight.data.copy_(
							self.GMF_model.embed_item_GMF.weight)
			self.embed_user_MLP.weight.data.copy_(
							self.MLP_model.embed_user_MLP.weight)
			self.embed_item_MLP.weight.data.copy_(
							self.MLP_model.embed_item_MLP.weight)

			# mlp layers
			for (m1, m2) in zip(
				self.MLP_layers, self.MLP_model.MLP_layers):
				if isinstance(m1, nn.Linear) and isinstance(m2, nn.Linear):
					m1.weight.data.copy_(m2.weight)
					m1.bias.data.copy_(m2.bias)

			# predict layers
			predict_weight = torch.cat([
				self.GMF_model.predict_layer.weight, 
				self.MLP_model.predict_layer.weight], dim=1)
			precit_bias = self.GMF_model.predict_layer.bias + \
						self.MLP_model.predict_layer.bias

			self.predict_layer.weight.data.copy_(0.5 * predict_weight)
			self.predict_layer.bias.data.copy_(0.5 * precit_bias)

	def forward(self, user, item):
		if not self.model == 'MLP':
			embed_user_GMF = self.embed_user_GMF(user)
			embed_item_GMF = self.embed_item_GMF(item)
			output_GMF = embed_user_GMF * embed_item_GMF
		if not self.model == 'GMF':
			embed_user_MLP = self.embed_user_MLP(user)
			embed_item_MLP = self.embed_item_MLP(item)
			interaction = torch.cat((embed_user_MLP, embed_item_MLP), -1)
			output_MLP = self.MLP_layers(interaction)

		if self.model == 'GMF':
			concat = output_GMF
		elif self.model == 'MLP':
			concat = output_MLP
		else:
			concat = torch.cat((output_GMF, output_MLP), -1)

		prediction = self.predict_layer(concat)
		return prediction.view(-1)


# main:

In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 125 kB 5.3 MB/s 


In [ ]:
import os
import time
import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
from tensorboardX import SummaryWriter

'''
import model
import config
import evaluate
import data_utils
'''

'\nimport model\nimport config\nimport evaluate\nimport data_utils\n'

In [ ]:
parser={}
parser["lr"] = 0.001          #help="learning rate"
parser["dropout"] = 0.0       #help="dropout rate"
parser["batch_size"] = 256    #help="batch size for training"
parser["epochs"] = 20         #help="training epoches"
parser["top_k"] = 10          #help="compute metrics@top_k"
parser["factor_num"] = 32     #help="predictive factors numbers in the model"
parser["num_layers"] = 3      #help="number of layers in MLP model"
parser["num_ng"] = 4          #help="sample negative items for training"
parser["test_num_ng"] = 99    #help="sample part of negative items for testing"
parser["out"] = True          #help="save model or   not"
parser["gpu"] = "0"           #help="gpu card ID"

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = parser["gpu"]
cudnn.benchmark = True

In [ ]:
!pwd

/content/drive/MyDrive/Seminar


In [ ]:
############################## PREPARE DATASET ##########################

train_data, test_data, user_num ,item_num, train_mat = load_all()

# construct the train and test datasets
train_dataset = NCFData(
		train_data, item_num, train_mat, parser["num_ng"], True)
test_dataset = NCFData(
		test_data, item_num, train_mat, 0, False)
train_loader = data.DataLoader(train_dataset,
		batch_size=parser["batch_size"], shuffle=True, num_workers=4)
test_loader = data.DataLoader(test_dataset,
		batch_size=parser["test_num_ng"]+1, shuffle=False, num_workers=0)

100%|██████████| 1000/1000 [00:00<00:00, 66859.61it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
########################### CREATE MODEL #################################
if config.model == 'NeuMF-pre':
	assert os.path.exists(config.GMF_model_path), 'lack of GMF model'
	assert os.path.exists(config.MLP_model_path), 'lack of MLP model'
	GMF_model = torch.load(config.GMF_model_path)
	MLP_model = torch.load(config.MLP_model_path)
else:
	GMF_model = None
	MLP_model = None

model = NCF(user_num, item_num, parser["factor_num"], parser["num_layers"], 
						parser["dropout"], config.model, GMF_model, MLP_model)
model.cuda()
loss_function = nn.BCEWithLogitsLoss()

if config.model == 'NeuMF-pre':
	optimizer = optim.SGD(model.parameters(), lr=lr)
else:
	optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
writer = SummaryWriter() # for visualization

In [ ]:
from tqdm import tqdm

In [ ]:
########################### TRAINING #####################################
count, best_hr = 0, 0
for epoch in range(parser["epochs"]):
	model.train() # Enable dropout (if have).
	start_time = time.time()
	train_loader.dataset.ng_sample()
	for user, item, label in tqdm(train_loader):
		try:
			user = user.cuda()
			item = item.cuda()
			label = label.float().cuda()

			model.zero_grad()
			prediction = model(user, item)
			loss = loss_function(prediction, label)
			loss.backward()
			optimizer.step()
			# writer.add_scalar('data/loss', loss.item(), count)
			count += 1
		except:
			break


	model.eval()
	HR, NDCG = metrics(model, test_loader, parser["top_k"])

	elapsed_time = time.time() - start_time
	print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
			time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
	print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

	if HR > best_hr:
		best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
		if parser["out"]:
			if not os.path.exists(config.model_path):
				os.mkdir(config.model_path)
			torch.save(model, 
				'{}{}.pth'.format(config.model_path, config.model))

print("End. Best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(
									best_epoch, best_hr, best_ndcg))


  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 20/20 [00:02<00:00,  7.27it/s]


The time elapse of epoch 000 is: 00: 00: 02
HR: 0.200	NDCG: 0.086


100%|██████████| 20/20 [00:02<00:00,  7.07it/s]


The time elapse of epoch 001 is: 00: 00: 03
HR: 0.200	NDCG: 0.075


100%|██████████| 20/20 [00:02<00:00,  7.06it/s]


...

The time elapse of epoch 007 is: 00: 00: 02
HR: 0.100	NDCG: 0.033


100%|██████████| 20/20 [00:02<00:00,  7.64it/s]


The time elapse of epoch 008 is: 00: 00: 02
HR: 0.100	NDCG: 0.033


  0%|          | 0/20 [00:00<?, ?it/s]Exception ignored in: <function _releaseLock at 0x7f988ea7b8c0>
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
  0%|          | 0/20 [00:05<?, ?it/s]


RuntimeError: ignored